In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"


import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

/home/kaoara/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-21 14:52:12,878	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Load the model and tokenizer

checkpoint = 'beomi/Llama-3-KoEn-8B'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir="/data/llmlaw/base_model")
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=5000)
vllm_model = LLM(model='beomi/Llama-3-KoEn-8B', tensor_parallel_size=4)
# model = AutoModelForCausalLM.from_pretrained(checkpoint, cache_dir="/data/llmlaw/base_model")
# model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map='', cache_dir="/data/llmlaw/base_model")

2024-08-21 14:52:16,539	INFO worker.py:1781 -- Started a local Ray instance.


INFO 08-21 14:52:18 config.py:623] Defaulting to use mp for distributed inference
INFO 08-21 14:52:18 llm_engine.py:161] Initializing an LLM engine (v0.5.0) with config: model='beomi/Llama-3-KoEn-8B', speculative_config=None, tokenizer='beomi/Llama-3-KoEn-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=beomi/Llama-3-KoEn-8B)
(VllmWorkerProcess pid=3225760) INFO 08-21 14:52:21 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=3225759) INFO 08-21 14:52:21 multiproc_worker_utils.py:215] Worker ready; awaiting

Traceback (most recent call last):
  File "/home/kaoara/anaconda3/envs/test/lib/python3.9/multiprocessing/resource_tracker.py", line 201, in main
    cache[rtype].remove(name)
KeyError: '/psm_3abea110'
Traceback (most recent call last):
  File "/home/kaoara/anaconda3/envs/test/lib/python3.9/multiprocessing/resource_tracker.py", line 201, in main
    cache[rtype].remove(name)
KeyError: '/psm_3abea110'
Traceback (most recent call last):
  File "/home/kaoara/anaconda3/envs/test/lib/python3.9/multiprocessing/resource_tracker.py", line 201, in main
    cache[rtype].remove(name)
KeyError: '/psm_3abea110'


INFO 08-21 14:52:22 weight_utils.py:218] Using model weights format ['*.safetensors']
(VllmWorkerProcess pid=3225760) INFO 08-21 14:52:22 weight_utils.py:218] Using model weights format ['*.safetensors']
(VllmWorkerProcess pid=3225758) INFO 08-21 14:52:22 weight_utils.py:218] Using model weights format ['*.safetensors']
(VllmWorkerProcess pid=3225759) INFO 08-21 14:52:22 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 08-21 14:52:24 model_runner.py:159] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=3225760) INFO 08-21 14:52:24 model_runner.py:159] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=3225758) INFO 08-21 14:52:25 model_runner.py:159] Loading model weights took 3.7417 GB
(VllmWorkerProcess pid=3225759) INFO 08-21 14:52:25 model_runner.py:159] Loading model weights took 3.7417 GB
INFO 08-21 14:52:26 distributed_gpu_executor.py:56] # GPU blocks: 70556, # CPU blocks: 8192
INFO 08-21 14:52:30 model_runner.py:878] Capturing the model

In [5]:
# Load the data
data_path = '/home/kaoara/dps/legal-project-data/DOMAIN_Preprocessed_CASE_LAW/sampled_100.jsonl'
with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

results = []

In [9]:
# Process data, tokenize, and split into input and label
batch_size = 2  # Adjust the batch size based on your GPU memory
tokenizer.pad_token = tokenizer.eos_token

def process_batch(batch):
    vllm_input_texts = [item['text'][:300] for item in batch]

    label_texts = [item['text'][300:] if len(tokenizer.tokenize(item['text'][300:])) <= 5000 
                   else tokenizer.decode(tokenizer.encode(item['text'][300:], truncation=True, max_length=5000), skip_special_tokens=True)
                   for item in batch]
    
    # input_texts = [item['text'] if len(tokenizer.tokenize(item['text'])) <= 5100 
    #                else tokenizer.decode(tokenizer.encode(item['text'])[:5100], skip_special_tokens=True)
    #                for item in batch]
    
    # input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).input_ids
    
    # input_ids = input_ids.to(model.device)

    # Generate output using the model
    generated_outputs = vllm_model.generate(vllm_input_texts, sampling_params)

    # Decode generated text
    generated_texts = [output.outputs[0].text for output in generated_outputs]

    # Calculate perplexity (PPL)
    # with torch.no_grad():
    #     outputs = model(input_ids=input_ids)
    #     log_likelihood = outputs.loss
    #     ppl = torch.exp(log_likelihood)

    for i, item in enumerate(batch):
        results.append({
            'text': item['text'],
            'tokenized_output': tokenizer.tokenize(item['text']),
            'label': label_texts[i],
            'input': item['text'][:300],
            'generated_output': generated_texts[i],
            # 'ppl': ppl[i].item()
        })

In [10]:
# Batch processing
for i in tqdm(range(0, len(data), batch_size)):
    batch = data[i:i+batch_size]
    process_batch(batch)

100%|██████████| 50/50 [44:40<00:00, 53.61s/it]


In [22]:
results[-1]['input']

'판시사항 \n 소재지관서의 증명이 없는 농지매매계약의 채권적 효력\n주문 \n 원판결을 파기한다.\n사건을 대전지방법원 합의부에 환송한다.\n이유 \n 그러면 원심은 원고가 원피고 사이의 본건 농지매매의 채권계약이 피고의 책임에 돌아갈 사유로 이행불능에 빠지게 된 것이니 이행에 갈음할 손해배상을 청구함이 원고의 소송취지에 의하여 명백한 이상 원심은 마땅히 농지매매의 채권계약이 성립된 것이고 비록 소재지 관서의 농지증명이 없을 지라도 과연 원고주장과 같이 매도인인 피고의 책임에 돌아갈 사유로 채권계약을 이행할 수 없게 된 것인가를 심구하여 판단'

In [17]:
results[-1]['generated_output']

'하였어야 할 것인데도 불구하고 그 농지증명이 없음을 이유로 농지매매의 채권계약이 성립되지 않았다고 판단하였음은 부당하다 할 것이니 이점을 지적하는 논지는 이유있다.\n그러므로 원판결을 파기하고 사건을 원심법원에 환송하기로 하여 관여법관의 일치된 의견으로 주문과 같이 판결한다.'

In [20]:
results[-1]['label']

'하여야 함에도 불구하고 만연히 소재지관서의 농지증명이 없는 이상 매매계약은 아직 효력이 발생 할 수 없다 하여 원고의 청구를 쉽사리 배척하였음은 농지 매매에 있어서의 소재지관서의 증명을 요하는 의 법의를 잘못 이해함으로써 심리를 다하지 않은 위법이 있다 할 것이며 원판결에 위에 말한 위법이 있다는 취지의 피고 소송대리인의 상고논지는 이유있다.\n그러므로 원판결을 파기하고 원심으로 하여금 다시 심리 재판하게 하기 위하여 관여한 법관 전원의 일치된 의견으로 주문과 같이 판결한다.'

In [25]:
# Save the results to a JSONL file
output_path = f"{checkpoint.split('/')[-1]}_results.jsonl"
with open(output_path, 'w', encoding='utf-8') as f:
    for result in results:
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

### PPL

In [7]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"


import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

/home/kaoara/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-22 07:48:53,362	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
# Load the model and tokenizer

checkpoint = 'beomi/Llama-3-KoEn-8B'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir="/data/llmlaw/base_model")
# sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=5000)
# vllm_model = LLM(model='beomi/Llama-3-KoEn-8B', tensor_parallel_size=4)
# model = AutoModelForCausalLM.from_pretrained(checkpoint, cache_dir="/data/llmlaw/base_model")
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map='auto', cache_dir="/data/llmlaw/base_model")

Loading checkpoint shards: 100%|██████████| 6/6 [00:10<00:00,  1.72s/it]


In [9]:
# Load the data
data_path = '/home/kaoara/dps/legal-project-data/DOMAIN_Preprocessed_CASE_LAW/sampled_100.jsonl'
with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

results = []

In [10]:
# Process data, tokenize, and split into input and label
batch_size = 4  # Adjust the batch size based on your GPU memory
tokenizer.pad_token = tokenizer.eos_token

def process_batch(batch):
    input_texts = [
        item['text'] if len(tokenizer.tokenize(item['text'])) <= 5000  # Redundant
        else tokenizer.decode(tokenizer.encode(item['text'], truncation=True, max_length=5100), skip_special_tokens=True)
        for item in batch
    ]

    # Tokenize inputs with padding and truncation
    input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).input_ids
    input_ids = input_ids.to(model.device)

    # Calculate loss for each sample individually
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        logits = outputs.logits

        # Initialize a list to store PPL for each sample
        ppl_list = []

        # Iterate through each sample in the batch
        for i in range(batch_size):
            # Compute loss for the individual sample
            shift_logits = logits[i, :-1, :].contiguous()
            shift_labels = input_ids[i, 1:].contiguous()

            loss_fct = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=tokenizer.pad_token_id)
            individual_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            # Calculate PPL for the individual sample
            non_pad_tokens = (shift_labels != tokenizer.pad_token_id).sum()
            ppl = torch.exp(individual_loss / non_pad_tokens)
            ppl_list.append(ppl.item())

    # Record results
    for i, item in enumerate(batch):
        results.append({
            'text': item['text'],
            'input': input_texts[i],
            'ppl': ppl_list[i]
        })


In [11]:
# Batch processing
for i in tqdm(range(0, len(data), batch_size)):
    batch = data[i:i+batch_size]
    process_batch(batch)

  0%|          | 0/25 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
100%|██████████| 25/25 [03:24<00:00,  8.17s/it]


In [13]:
# Save the results to a JSONL file
output_path = f"{checkpoint.split('/')[-1]}_ppl.jsonl"
with open(output_path, 'w', encoding='utf-8') as f:
    for result in results:
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

In [12]:
def calculate_average_ppl(results):
    if not results:
        return 0.0
    total_ppl = sum(item['ppl'] for item in results)
    average_ppl = total_ppl / len(results)
    return average_ppl

# Example usage
# Assuming you've already processed batches and populated the `results` list
average_ppl = calculate_average_ppl(results)
print(f"Average PPL: {average_ppl}")

Average PPL: 3.0854262924194336
